# Case study for KIP project

In [ ]:
import case_study_lead_braking_2d

In [ ]:
%debug

In [ ]:
from simulation import SimulationLeadBraking
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
%matplotlib inline

In [ ]:
s = SimulationLeadBraking(min_simulations=5)

In [ ]:
s.simulation(dict(v0=20, amean=2, dv=20), plot=True, seed=2)

In [ ]:
# Parameters: (starting speed, average deceleration, speed difference)
s.get_probability(dict(v0=20, amean=2, dv=20), plot=True, seed=1)

In [ ]:
ameans = np.arange(0.5, 5.0, 0.1)
deltavs = np.arange(5, 20, 1)

In [ ]:
result = np.zeros((len(ameans), len(deltavs)))
for i, amean in enumerate(tqdm(ameans)):
    for j, deltav in enumerate(deltavs):
        result[i, j] = s.get_probability((20, amean, deltav), seed=0)

In [ ]:
f, ax = plt.subplots(1, 1)
contourplot = ax.contourf(deltavs, ameans, result)
ax.set_ylabel("Mean deceleration [m/s$^2$]")
ax.set_xlabel("Speed difference [m/s]")
cbar = f.colorbar(contourplot)
cbar.ax.set_ylabel("Probability of collision")

In [ ]:
import case_study_risk_paper

In [ ]:
%debug

In [ ]:
import os
from typing import Tuple
import time
import collections
import matplotlib.pyplot as plt
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm
from domain_model import Scenario, BSplines
from stats.fastkde import KDE
from databaseemulator import DataBaseEmulator

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_scenarios", "ego_braking.json")
DBE = DataBaseEmulator(filename)
print("Number of scenarios: {:d}".format(len(DBE.collections["scenario"])))

## Get scenario parameters

In [ ]:
def parameters(scenario: Scenario) -> np.ndarray:
    """ Get the parameters of the ego-braking scenario. 
    
    It is assumed that the secnario contains only one scenario.
    This activity describes the braking activity of the ego
    vehicle. The following parameters are obtained:
    - Initial speed
    - Average deceleration
    - Speed difference
    
    :param scenario: The scenario describing the braking activity
        of the ego vehicle.
    :return: The mentioned parameters.
    """
    activity = scenario.activities[0]
    vstart_vend = activity.get_state(time=[activity.tstart, activity.tend])
    vstart = vstart_vend[0]
    vdiff = (vstart - vstart_vend[1])
    amean = vdiff / (activity.tend - activity.tstart)
    return np.array([vstart, amean, vdiff])

In [ ]:
nscenarios = len(DBE.collections["scenario"])
pars = np.array([parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)])

In [ ]:
plt.plot(pars[:, 2]*3.6, pars[:, 1], '.')
plt.xlabel("Speed difference [km/h]")
plt.ylabel("Mean deceleration [m/s$^2$]")

## Estimate probability density function

In [ ]:
pars_std = np.std(pars, axis=0)
pars_mean = np.mean(pars, axis=0)
pars_scaled = (pars - pars_mean) / pars_std
kde = KDE(pars_scaled)
_ = kde.compute_bandwidth()

In [ ]:
ameans = np.linspace(0, 4, 50)
speeds = np.linspace(10, 110, 11)
x, y = np.meshgrid(ameans, speeds)
xpdf = np.concatenate((x[:, :, np.newaxis], 
                       y[:, :, np.newaxis],
                       np.ones_like(x)*1000), axis=2)
cdf = kde.cdf(xpdf)

In [ ]:
def sample(vinit: float) -> Tuple[float, float]:
    """ Sample the remaining two parameters of the KDE, given the first parameter.
    
    :param vinit: The initial speed.
    :return: Average deceleration and speed difference. 
    """
    #a = np.random.rand()
    b = np.random.rand()
    c = np.random.rand()
    
    lim = 10
    n = 200
    xmar = np.linspace(-lim, lim, n)
    xone = np.ones(n)
    #ycdf = kde.cdf(np.vstack((xmar, xone*lim, xone*lim)).T)
    #x1 = np.interp(a, ycdf, xmar)
    x1 = vinit
    
    ycdf = (kde.cdf(np.vstack((xone*(x1+1e-4), xmar, xone*lim)).T) -
            kde.cdf(np.vstack((xone*x1, xmar, xone*lim)).T)) / 1e-4
    x2 = np.interp(b*ycdf[-1], ycdf, xmar)
    
    ycdf = (kde.cdf(np.vstack((xone*(x1+1e-4), xone*(x2+1e-4), xmar)).T) -
            kde.cdf(np.vstack((xone*(x1+1e-4), xone*(x2-1e-4), xmar)).T) -
            kde.cdf(np.vstack((xone*(x1-1e-4), xone*(x2+1e-4), xmar)).T) +
            kde.cdf(np.vstack((xone*(x1-1e-4), xone*(x2-1e-4), xmar)).T)) / (4*1e-8)
    x3 = np.interp(c*ycdf[-1], ycdf, xmar)
    
    return (x2, x3)

## Run a single simulation

In [ ]:
class IDM:
    def __init__(self, a, b, delta, s0, T, amin, tr):
        self.a = a
        self.b = b
        self.delta = delta
        self.s0 = s0
        self.T = T
        self.amin = amin
        self.tr = tr
        
        self.t, self.v, self.x = 0, 0, 0
        self.v0 = 0
        
        self.times = collections.deque(maxlen=int(self.tr*200))
        self.accelerations = collections.deque(maxlen=len(self.times))
    
    def init_simulation(self, x0, v0):
        self.times = collections.deque(maxlen=int(self.tr*100))
        self.accelerations = collections.deque(maxlen=int(self.tr*100))
        self.times.append(-1000)
        self.times.append(-0.01)
        self.accelerations.append(0)
        self.accelerations.append(0)
        self.t, self.v, self.x = 0, v0, x0
        self.v0 = v0*10
    
    def step_simulation(self, t, xlead, vlead):
        # Update speed
        a = np.max((self.amin, np.interp(t-self.tr, self.times, self.accelerations)))
        self.v += a*(t - self.t)
        
        # Update position
        self.x += self.v*(t - self.t)
        
        # Calculate acceleration based on IDM
        sstar = self.s0 + self.v*self.T + self.v*(self.v-vlead)/(2*np.sqrt(self.a*self.b))
        self.accelerations.append(self.a*(1 - (self.v/self.v0)**self.delta -
                                          (sstar / (xlead - self.x))**2))
        self.times.append(t)
        self.t = t
        
        return self.x, self.v

In [ ]:
class leader:
    def __init__(self, tconst):
        self.x0, self.v0 = 0, 0
        self.dv, self.T = 0, 0
        self.tconst = tconst
        
    def init_simulation(self, x0, v0, amean, dv):
        self.dv = dv
        self.T = dv / amean
        self.x0, self.v0 = x0, v0
        
    def step_simulation(self, t):
        if t <= self.tconst:
            speed = self.v0
            distance = self.x0 + self.v0*t
        elif t < self.tconst + self.T:
            speed = self.v0 - self.dv/2*(1 - np.cos(np.pi*(t - self.tconst) / self.T))
            distance = self.x0 + self.v0*t - \
                self.dv/2*(t-self.tconst - self.T/np.pi*np.sin(np.pi*(t-self.tconst)/self.T))
        else:
            speed = self.v0 - self.dv
            distance = self.x0 + self.dv*(self.T/2+self.tconst) + (self.v0-self.dv)*t
        return distance, speed        

In [ ]:
L = leader(5)
F = IDM(0.73, 1.67, 4, 2, 1, -3, 0.5)

def simulation(v0, amean, dv):
    L.init_simulation(F.s0+v0*F.T, v0, amean, dv)
    F.init_simulation(0, v0)
    t = 0
    prev_dist = 0
    xl, xf = 1, 0
    while t < 10 or prev_dist > xl - xf:
        prev_dist = xl - xf
        t += 0.01
        xl, vl = L.step_simulation(t)
        xf, vf = F.step_simulation(t, xl, vl)
        
        if xf > xl:
            # Collision!
            return 1
    return 0

In [ ]:
v0 = 30
amean = 2
dv = 20
simulation(v0, amean, dv)

In [ ]:
L.init_simulation(2+v0*1, v0, amean, dv)
F.init_simulation(0, v0)
t = np.arange(0.01, 20, 0.01)
xl = np.zeros_like(t)
vl = np.zeros_like(t)
xf = np.zeros_like(t)
vf = np.zeros_like(t)
for i, time in enumerate(t):
    xl[i], vl[i] = L.step_simulation(time)
    xf[i], vf[i] = F.step_simulation(time, xl[i], vl[i])

In [ ]:
plt.plot(t, vl*3.6, label="Leader")
plt.plot(t, vf*3.6, label="Ego")
plt.xlabel("Time [s]")
plt.ylabel("Speed [km/h]")
_ = plt.legend()

In [ ]:
plt.plot(t, xl - xf)
plt.xlabel("Time [s]")
_ = plt.ylabel("Distance")

In [ ]:
plt.plot(t, np.gradient(vl) / np.gradient(t), label="Leader")
plt.plot(t, np.gradient(vf) / np.gradient(t), label="Ego")
plt.xlabel("Time [s]")
plt.ylabel("Deceleration [m/s$^2$]")
_ = plt.legend()

In [ ]:
speeds = np.linspace(10, 30, 21)
ameans = np.linspace(0.1, 5, 50)
deltavs = np.linspace(0, 30, 16)
thws = np.linspace(0.1, 1.5, 15)

In [ ]:
result = np.zeros((len(thws), len(speeds), len(deltavs), len(ameans)), dtype=np.bool)

In [ ]:
filename = os.path.join("data", "KIP result.p")
if os.path.exists(filename):
    with open(filename, 'rb') as file:
        result = pickle.load(file)
else:
    for i, thw in enumerate(tqdm(thws)):
        F = IDM(0.73, 1.67, 4, 2, thw, -3, 0.5)
        for j, speed in enumerate(tqdm(speeds, leave=False)):
            for k, deltav in enumerate(deltavs):
                if deltav > speed:
                    continue
                for m, amean in enumerate(ameans):
                    result[i, j, k, m] = simulation(speed, amean, deltav)
    with open(filename, 'wb') as file:
        pickle.dump(result, file)

In [ ]:
r = np.zeros((len(thws), len(speeds)))
for i in tqdm(range(len(thws))):
    for j, speed in enumerate(speeds):
        p = 0
        bb, cc = np.meshgrid(ameans, deltavs[deltavs <= speed])
        aa = np.ones_like(bb) * speed
        samples = np.concatenate((aa[:, :, np.newaxis], 
                                  bb[:, :, np.newaxis], 
                                  cc[:, :, np.newaxis]), axis=2)
        probs = kde.score_samples((samples - pars_mean) / pars_std)
        r[i, j] = np.sum(probs * result[i][j][deltavs <= speed]) / np.sum(probs)

In [ ]:
fig, ax = plt.subplots(1, 1)
cs = plt.contourf(thws, speeds*3.6, r.T)
plt.xlabel("Time headway [s]")
plt.ylabel("Speed [km/h]")
plt.title("Likelihood of collision if preceding vehicle brakes.")
fig.colorbar(cs, ax=ax)